In [1]:
from src.data_loader.data_loader import BooksDataset
from src.models.mm_model import MmModel
import pandas as pd
from src.train import Trainer
import torch

In [2]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [3]:
dataset = BooksDataset(data_dir="../data/books")

In [4]:
model = MmModel(n_users=dataset.n_users,n_items=dataset.n_items,adjacency_matrix=dataset.get_dataset("adjacency_matrix"),embed_size=64,n_layers=3)
model.to(device)

MmModel(
  (E0): Embedding(48752, 64)
)

In [5]:
trainer = Trainer(model=model,dataset=dataset,lr=0.005)

In [6]:
trainer.train(epochs=10,batch_size=1024)

100%|██████████| 1/1 [00:17<00:00, 17.99s/it]


{'precision': array([0.00915 , 0.007525, 0.00577 ]), 'recall': array([0.01734349, 0.02958431, 0.05241878]), 'ndcg': array([0.03906762, 0.0491048 , 0.06690311]), 'hit_ratio': array([0.08 , 0.115, 0.176]), 'auc': 0.0}
Epoch 0 Loss 0.4149689642712474 Time 19.725087881088257


100%|██████████| 1/1 [00:18<00:00, 18.07s/it]


{'precision': array([0.0093  , 0.007525, 0.00578 ]), 'recall': array([0.01740971, 0.02958431, 0.05291878]), 'ndcg': array([0.03927314, 0.04913443, 0.06702069]), 'hit_ratio': array([0.0805, 0.115 , 0.1765]), 'auc': 0.0}
Epoch 1 Loss 0.07198358078797658 Time 19.842944622039795


100%|██████████| 1/1 [00:19<00:00, 19.95s/it]


...

{'precision': array([0.00925 , 0.007425, 0.0058  ]), 'recall': array([0.01683006, 0.02878499, 0.05268802]), 'ndcg': array([0.03925259, 0.04865712, 0.06703569]), 'hit_ratio': array([0.08  , 0.1135, 0.1765]), 'auc': 0.0}
Epoch 7 Loss 0.08030623445908229 Time 21.209127187728882


100%|██████████| 1/1 [00:19<00:00, 19.31s/it]


{'precision': array([0.00925, 0.0074 , 0.00579]), 'recall': array([0.01683006, 0.02878051, 0.05274356]), 'ndcg': array([0.03924792, 0.04865231, 0.06704506]), 'hit_ratio': array([0.08  , 0.1135, 0.1765]), 'auc': 0.0}
Epoch 8 Loss 0.08755200554927191 Time 20.8336021900177


100%|██████████| 1/1 [00:20<00:00, 20.00s/it]

{'precision': array([0.0092  , 0.007425, 0.00581 ]), 'recall': array([0.01764949, 0.02884908, 0.05275677]), 'ndcg': array([0.03915367, 0.04864751, 0.06701804]), 'hit_ratio': array([0.08  , 0.1135, 0.1765]), 'auc': 0.0}
Epoch 9 Loss 0.07683867464462917 Time 21.728339672088623


In [5]:
device

device(type='cuda')

In [15]:
# create a tenor with 3 items
item = torch.tensor([0,2,3]).to(device)
user = torch.tensor([0,2,3]).to(device)
torch.mm(user,item.view(-1,1))





RuntimeError: self must be a matrix

In [13]:
import numpy as np
def precision_at_k(r, k):
    """Score is precision @ k
    Relevance is binary (nonzero is relevant).
    Returns:
        Precision @ k
    Raises:
        ValueError: len(r) must be >= k
    """
    assert k >= 1
    r = np.asarray(r)[:k]
    return np.mean(r)
# test
r = [0,1,1,0,0,1,0,0,0,1]
precision_at_k(r, 5)

0.4

In [27]:
def recall(rank, ground_truth, N):
    return len(set(rank[:N]) & set(ground_truth)) / float(len(set(ground_truth)))
# test
rank = [0,1,1,0,0,1,0,0,0,1]
ground_truth = [0]
recall(rank, ground_truth, 5)

1.0

In [28]:
set(rank[:5]) & set(ground_truth) # this operation is the intersection of the two sets

{0}

In [9]:
import multiprocessing
multiprocessing.cpu_count()


12

In [2]:
import heapq
Ks = [5]

def rank_list_by_heapq(rating, user_pos_test, test_items):
    item_score = {}
    for i in test_items:
        item_score[i] = rating[i]

    K_max = max(Ks)
    K_max_item_score = heapq.nlargest(K_max, item_score, key=item_score.get)

    r = []
    for i in K_max_item_score:
        if i in user_pos_test:
            r.append(1)
        else:
            r.append(0)
    auc = 0.
    return r, auc

rating = [0.9,0.2,0.3,0.4,0.5,0.6,0.7,0.8]
user_pos_test = [0,1,2]
test_items = [0,1,2,3,4,5,6,7]
rank_list_by_heapq(rating, user_pos_test, test_items)

([1, 0, 0, 0, 0], 0.0)

In [26]:
item_score = {}
for i in test_items:
    item_score[str(i)] = rating[i]


In [27]:
item_score

{'0': 0.9,
 '1': 0.2,
 '2': 0.3,
 '3': 0.4,
 '4': 0.5,
 '5': 0.6,
 '6': 0.7,
 '7': 0.8}

In [30]:
import pandas as pd

#create data frame with colmuns as items keys and values as scores
df = pd.DataFrame(item_score,index=[0])
df


,0,1,2,3,4,5,6,7
0,0.9,0.2,0.3,0.4,0.5,0.6,0.7,0.8


In [15]:
items

,item,score
0,0,0.9
1,1,0.2
2,2,0.3
3,3,0.4
4,4,0.5
5,5,0.6
6,6,0.7
7,7,0.8


In [32]:
# convert data frame to dict 
items = df.to_dict(orient="records")
items[0]

{'0': 0.9,
 '1': 0.2,
 '2': 0.3,
 '3': 0.4,
 '4': 0.5,
 '5': 0.6,
 '6': 0.7,
 '7': 0.8}